In [1]:
from __future__ import division, print_function
import sys, os, glob, time, warnings, gc
# import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack, hstack
import fitsio
from astropy.io import fits
import healpy as hp
from astropy import wcs

from scipy.ndimage.filters import gaussian_filter
from pathlib import Path
from multiprocessing import Pool

from scipy import stats

In [2]:
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'figure.facecolor':'w'} 
plt.rcParams.update(params)

In [3]:
image_dir = '/global/cfs/cdirs/cosmo/staging/'
surveyccd_path = '/global/project/projectdirs/cosmo/work/legacysurvey/dr9m/survey-ccds-90prime-dr9.fits.gz'

# image_path_list = glob.glob(os.path.join(image_dir, '*ooi*.fits.fz'))
ccd = Table(fitsio.read(surveyccd_path))

In [4]:
mask = ccd['ccd_cuts']==0
ccd = ccd[mask]

In [5]:
# Only keep unique exposures
_, idx = np.unique(ccd['expnum'], return_index=True)
ccd = ccd[idx]

In [6]:
ccdnum_list = [1, 2, 3, 4]
ccd_ra = [0.2813, 0.2813, -0.2813, -0.2813]
ccd_dec = [0.263, -0.263, 0.263, -0.263]

binsize = 4
pix_size = 0.454/3600*binsize
vrange = 1.

In [7]:
mask = np.array(['ksb_160604_055226' in tmp for tmp in ccd['image_filename']])
print(np.sum(mask))
ccd_index = np.where(mask)[0][0]
print(ccd['expnum'][ccd_index], ccd_index)

1
75430073 9083


In [8]:
mask = np.array(['CP20160603' in tmp for tmp in ccd['image_filename']])
print(np.sum(mask))
idx = np.where(mask)[0]

157


In [9]:
ccd[mask]

image_filename,image_hdu,camera,expnum,plver,procdate,plprocid,ccdname,object,propid,filter,exptime,mjd_obs,airmass,fwhm,width,height,ra_bore,dec_bore,crpix1,crpix2,crval1,crval2,cd1_1,cd1_2,cd2_1,cd2_2,yshift,ra,dec,skyrms,sig1,ccdzpt,zpt,ccdraoff,ccddecoff,ccdskycounts,ccdskysb,ccdrarms,ccddecrms,ccdphrms,phrms,ccdnastrom,ccdnphotom,ccd_cuts
bytes120,int16,bytes7,int64,bytes8,bytes19,bytes7,bytes4,bytes35,bytes10,bytes1,float32,float64,float32,float32,int16,int16,float64,float64,float32,float32,float64,float64,float32,float32,float32,float32,bool,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int64
90prime/CP/V2.3/CP20160603/ksb_160604_035950_ooi_g_ls9.fits.fz,1,90prime,75430038,V2.3,2019-08-27T18:08:14,956f01c,CCD1,31112,BASS,g,250.4,57543.1665534,1.13,3.61,4032,4096,179.256958333,54.2526666667,4095.71,4283.46,179.256958333,54.2526666667,-4.55822e-07,-0.000125754,-0.000125789,6.30239e-07,False,179.743639559,54.512304304,0.168846,0.0129785,25.265,25.2676,-0.01053,-0.0120556,7.01614,23.1048,0.0319894,0.0427656,0.0204251,0.0195448,538,627,0
90prime/CP/V2.3/CP20160603/ksb_160604_040612_ooi_g_ls9.fits.fz,1,90prime,75430039,V2.3,2019-08-27T18:08:12,956f01c,CCD1,33332,BASS,g,250.0,57543.1709789,1.11,3.83,4032,4096,190.79775,56.2517777778,4095.42,4283.23,190.79775,56.2517777778,-5.22153e-07,-0.000125764,-0.000125795,6.84569e-07,False,191.310028125,56.5111900919,0.155791,0.0115686,25.3212,25.3225,-0.0155089,-0.0128987,6.12358,23.2525,0.0313767,0.0450451,0.0153592,0.0180804,583,681,0
90prime/CP/V2.3/CP20160603/ksb_160604_041123_ooi_g_ls9.fits.fz,1,90prime,75430040,V2.3,2019-08-27T18:08:15,956f01c,CCD1,32222,BASS,g,247.6,57543.1745823,1.13,3.77,4032,4096,183.825875,55.2523611111,4095.55,4283.37,183.825875,55.2523611111,-4.8183e-07,-0.00012576,-0.00012579,6.57114e-07,False,184.324943749,55.5118779302,0.156474,0.012175,25.2796,25.2909,-0.0236535,-0.00944827,6.13092,23.2512,0.0388184,0.0432434,0.0165053,0.0211093,582,656,0
90prime/CP/V2.3/CP20160603/ksb_160604_044113_ooi_r_ls9.fits.fz,1,90prime,75430048,V2.3,2019-08-27T19:31:39,956f01c,CCD1,15663,BASS,r,250.9,57543.1952986,1.02,4.47,4032,4096,202.675791667,40.7524444444,4095.35,4282.65,202.675791667,40.7524444444,-5.33147e-07,-0.000125753,-0.000125769,6.97097e-07,False,203.05028754,41.0122129144,0.213569,0.0144171,25.4357,25.4216,-0.0183549,-0.06933,10.6025,22.7365,0.0280681,0.077136,0.0189999,0.0187607,650,754,0
90prime/CP/V2.3/CP20160603/ksb_160604_044621_ooi_r_ls9.fits.fz,1,90prime,75430049,V2.3,2019-08-27T19:31:39,956f01c,CCD1,14383,BASS,r,250.6,57543.1988567,1.01,4.41,4032,4096,205.786416667,39.7525277778,4095.34,4282.51,205.786416667,39.7525277778,-5.42252e-07,-0.000125752,-0.00012577,7.05611e-07,False,206.155367803,40.0123024554,0.212027,0.0140484,25.4514,25.4396,-0.0181745,-0.0519977,10.4129,22.7561,0.0295677,0.0629929,0.0186593,0.018217,661,773,0
90prime/CP/V2.3/CP20160603/ksb_160604_045136_ooi_r_ls9.fits.fz,1,90prime,75430050,V2.3,2019-08-27T19:31:43,956f01c,CCD1,15673,BASS,r,150.8,57543.2025048,1.02,4.57,4032,4096,204.015833333,40.7514444444,4095.29,4282.57,204.015833333,40.7514444444,-5.39312e-07,-0.00012575,-0.000125769,7.02733e-07,False,204.390314273,41.0111944707,0.262936,0.0171955,25.4617,25.452,-0.0137626,-0.0674373,10.1731,22.7814,0.0277305,0.0745094,0.0165336,0.0162947,637,711,0
90prime/CP/V2.3/CP20160603/ksb_160604_045503_ooi_r_ls9.fits.fz,1,90prime,75430051,V2.3,2019-08-27T19:31:40,956f01c,CCD1,14393,BASS,r,150.6,57543.2048999,1.02,3.01,4032,4096,207.1065,39.7504166667,4095.25,4282.55,207.1065,39.7504166667,-5.4982e-07,-0.000125761,-0.00012578,7.11118e-07,False,207.475489746,40.0101850726,0.255806,0.0163993,25.4913,25.4783,-0.0239523,-0.0168349,9.83233,22.8184,0.0285684,0.0313789,0.0169183,0.0166543,635,742,0
90prime/CP/V2.3/CP20160603/ksb_160604_045913_ooi_r_ls9.fits.fz,1,90prime,75430052,V2.3,2019-08-27T19:31:43,956f01c,CCD1,13093,BASS,r,88.7,57543.2077911,1.01,3.07,4032,4096,208.829791667

In [10]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603.pdf') as pdf:

    for ccd_index in idx[::5]:

        fn = ccd['image_filename'][ccd_index].strip()
        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

        ######################## Original mask ########################
        
        # mask_path = '/global/u2/r/rongpu/temp/90prime_junk/mask_{}.npz'.format(expnum)
        # img_mask = np.load(mask_path)
        
        fn_ood = fn.replace('_ooi_', '_ood_')
        mask_path = os.path.join(image_dir, fn_ood)
        
        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky
            
            # if 'CCD'+str(ccdnum) in img_mask.keys():
            #     img[img_mask['CCD'+str(ccdnum)]] = 0.
            ood = fits.getdata(mask_path, extname='ccd'+str(ccdnum))
            img[ood!=0] = 0.

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

9057 g 75430038
9062 r 75430050
9067 r 75430055
9072 r 75430062
9077 r 75430067
9082 r 75430072
9087 r 75430077
9092 r 75430083
9097 r 75430090
9102 r 75430095
9107 r 75430100
9112 r 75430105
9117 r 75430110
9122 r 75430115
9127 r 75430120
9132 r 75430125
9137 r 75430133
9142 r 75430138
9147 r 75430145
9152 r 75430150
9157 r 75430155
9162 r 75430160
9167 r 75430166
9172 r 75430171
9177 r 75430176
9182 r 75430181
9187 r 75430186
9192 r 75430192
9197 r 75430197
9202 r 75430202
9207 r 75430207
9212 r 75430212


In [34]:
# idx = np.where(ccd['expnum']==79240048)[0][0]
idx1 = np.concatenate([np.arange(idx.min()-6, idx.min()), np.arange(idx.max()+1, idx.max()+6)])
print(idx1)

from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('plots2/test_20160603_neighbors.pdf') as pdf:

    for ccd_index in idx1:

        # from matplotlib.backends.backend_pdf import PdfPages
        # with PdfPages('plots/90prime_images-ccd4.pdf') as pdf:
        # for ccd_index in range(idx-20, idx+20):

        expnum = ccd['expnum'][ccd_index]
        band = ccd['filter'][ccd_index]
        print(ccd_index, band, expnum)
        fn = ccd['image_filename'][ccd_index]

        plt.figure(figsize=(14, 12))

        for ii, ccdnum in enumerate(ccdnum_list):

            try:
                img = fits.getdata('/global/cfs/cdirs/cosmo/staging/'+fn, extname='CCD'+str(ccdnum))
            except:
                print('Failure loading {}'.format('/global/cfs/cdirs/cosmo/staging/'+fn))
                continue

            # naive sky estimation
            mask = (img<np.percentile(img.flatten(), 95))
            median_sky = np.median(img[mask].flatten())
            img = img - median_sky

            ################ downsize image ################

            trim_size_x = img.shape[1] % binsize
            trim_size_y = img.shape[0] % binsize
            img = img[:(img.shape[0]-trim_size_y), :(img.shape[1]-trim_size_x)]
            # to ignore NAN values, use np.nanmean
            img = np.nanmean(np.nanmean(img.reshape((img.shape[0]//binsize, binsize, img.shape[1]//binsize,-1)), axis=3), axis=1)

            ################################################

            ysize, xsize = img.shape        
            ra, dec = ccd_ra[ii], ccd_dec[ii]

            img[~np.isfinite(img)] = 0
            # img = gaussian_filter(img, 4, mode='reflect', truncate=3)
            fig = plt.imshow(img.T, cmap='seismic', vmin=-vrange, vmax=vrange, 
                       extent=(ra+ysize*pix_size/2, ra-ysize*pix_size/2, dec-xsize*pix_size/2, dec+xsize*pix_size/2))

        plt.axis([0.55, -0.55, -0.55, 0.55])
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)
        plt.colorbar(fraction=0.04, pad=0.04)
        plt.title('expnum={}  ccd_cuts={}  {}'.format(expnum, ccd['ccd_cuts'][ccd_index], fn))
        plt.tight_layout()
        # plt.savefig(os.path.join(plot_dir, os.path.basename(image_path).replace('.fits.fz', '.png')))
        pdf.savefig()
        plt.close()

[9051 9052 9053 9054 9055 9056 9214 9215 9216 9217 9218]
9051 g 75420170
9052 g 75420171
9053 g 75420172
9054 g 75420173
9055 g 75420174
9056 g 75420175
9214 r 75440042
9215 r 75440043
9216 r 75440044
9217 r 75440045
9218 r 75440046
